[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CCS-ZCU/EuPaC_shared/blob/master/NOSCEMUS_getting-started.ipynb)

This Jupyter notebook has been prepared for the EuPaC Hackathon and provides an easy way to start working with the NOSCEMUS dataset — no need to clone the entire repository or download additional data. It is fully compatible with cloud platforms like Google Colaboratory (click the badge above) and runs without requiring any specialized library installations.

As such, it is intended as a starting point for EuPaC participants, including those with minimal coding experience.

In [ ]:
import pandas as pd
import nltk
import re
import requests
import json
import io

In [ ]:
# Phase 1: Setup - Import Libraries
import folium
import geopandas as gpd
import os
import time

In [ ]:
noscemus_metadata = pd.read_csv("https://raw.githubusercontent.com/CCS-ZCU/noscemus_ETF/refs/heads/master/data/metadata_table_long.csv")
noscemus_metadata.head(5)

In [ ]:
# Phase 0: Data Exploration
# Display DataFrame Columns
print("Columns in noscemus_metadata:")
print(noscemus_metadata.columns.tolist())

# Replace 'candidate_column_name' with a column name from the list above
candidate_column_name = 'Place' # <-- CHANGE THIS VALUE 

if candidate_column_name in noscemus_metadata.columns:
    print(f"\nUnique values in '{candidate_column_name}':")
    # Display a sample of unique values and their counts
    print(noscemus_metadata[candidate_column_name].value_counts().head(30))
    print(f"\nNumber of unique values in '{candidate_column_name}': {noscemus_metadata[candidate_column_name].nunique()}")
    print(f"Number of missing values in '{candidate_column_name}': {noscemus_metadata[candidate_column_name].isnull().sum()}")
    # Show some raw examples of the data in this column
    print("\nSample raw entries (up to first 20 non-null):")
    print(noscemus_metadata[candidate_column_name].dropna().head(20).tolist())
else:
    print(f"Column '{candidate_column_name}' not found in DataFrame. Please choose from the list printed above.")

In [ ]:
# Phase 1: Data Extraction - Extract 'Place' column
actual_publication_place_column = 'Place'
places_series = noscemus_metadata[actual_publication_place_column].astype(str).str.strip()
unique_raw_places = places_series.unique()
print(f"Found {len(unique_raw_places)} unique raw place mentions from '{actual_publication_place_column}'.")
print("Sample of raw places (first 50):")
print(unique_raw_places[:50])

In [ ]:
# Phase 2: Cleaning Publication Places

def clean_place_name(name):
    name_str = str(name).lower().strip() # Ensure string, lowercase, strip whitespace

    # Remove content in brackets and parentheses
    name_str = re.sub(r"\(.*?\)", "", name_str).strip()
    name_str = re.sub(r"\[.*?\]", "", name_str).strip()

    # Specific known substitutions (expand as needed)
    replacements = {
        "lvgduni batavorvm": "leiden",
        "lugduni batavorum": "leiden",
        "amstelodami": "amsterdam",
        "amstelædami": "amsterdam",
        "amstelodamum": "amsterdam",
        "parisiis": "paris",
        "londini": "london",
        "franequerae": "franeker",
        "hafniae": "copenhagen",
        "coloniae agrippinae": "cologne",
        "antverpiae": "antwerp",
        "lipsiae": "leipzig",
        "argentorati": "strasbourg",
        "s.l.": "", # Sine loco (without place)
        "s. l.": "",
        "s.a.": "", # Sine anno (less relevant for place)
        "s.n.": "",  # Sine nomine (less relevant for place)
        "o.o.": "", # Ohne Ort (German for without place)
        "o. o.": ""
    }
    name_str = replacements.get(name_str, name_str)

    # For entries with multiple places like "Liegnitz, Wrocław", we'll try to take the first one for now.
    # A more advanced approach could be to split and geocode each, but that adds complexity.
    # Geonames might be able to handle some of these directly or find the primary city.
    if ',' in name_str:
        # Attempt to split by common delimiters like comma or semicolon
        parts = re.split(r'[,;/&]', name_str)
        # Take the first part, clean it further if necessary
        name_str = parts[0].strip()
        # Re-apply replacements if the first part is a known one
        name_str = replacements.get(name_str, name_str)
    
    # Remove most remaining non-alphanumeric characters, except hyphens and apostrophes within words
    name_str = re.sub(r"[^a-z\s'’-]", "", name_str, flags=re.UNICODE).strip()
    # Normalize multiple spaces to single space
    name_str = re.sub(r"\s+", " ", name_str).strip()

    return name_str.title() if name_str else "" # Title case for Geonames, or empty if cleaning resulted in nothing

# Apply the cleaning function
# Handle potential NaN values in 'places_series' by filling them with an empty string before applying
cleaned_places_list = sorted(list(set(places_series.fillna('').apply(clean_place_name).tolist())))
cleaned_places_list = [p for p in cleaned_places_list if p] # Remove empty strings that might result from cleaning

print(f"Found {len(cleaned_places_list)} unique cleaned place names after initial cleaning.")
print("Sample of cleaned places (first 50):")
print(cleaned_places_list[:50])

All mapping between the metadata and the actual textual data happens through the "id" column.
Thus, knowing its ID, you can load full textual data (both raw and morphologically annotated) any text or a subset of texts.

In [ ]:
id = 1378359
base_url = "https://ccs-lab.zcu.cz/noscemus_sents_data/{}.json"
sents_data = json.load(io.BytesIO(requests.get(base_url.format(str(id))).content))

In [ ]:
# the sents_data is a list of sentences from the given document
# in addition to the raw text of the sentence, it also contains the lemmatized tokens and their POS tags
# look at first few sentences to get an idea of the format:
sents_data[110:115]

For each sentence, you see the following elements:
* (1) ID of the source document
* (2) index of the sentence (remember that Python's indexing starts with 0)
* (2) token data for the sentence

The token data for each token contain:
   * (a) The token as it is in the sentence
   * (b) The automatically assigned lemma corresponding to the token
   * (c) Its Part-of-Speech
   * (d) Its starting positional index within the sentence
   * (e) Its ending positional index within the sentence

In [ ]:
# if you want a raw text of the document, use the following:
rawtext = " ".join([sent_data[2] for sent_data in sents_data])
rawtext[:1000]

In [ ]:
# if you want a list of lemmatized tokens, filtered by certain POS-tags, use the following:
lemmatized_sents = []
for sent_data in sents_data:
    lemmatized_sent = []
    for token in sent_data[3]:
        if token[2] in ["NOUN", "VERB", "ADJ", "PROPN"]:
            lemmatized_sent.append(token[0])
    lemmatized_sents.append(lemmatized_sent)
lemmatized_sents[150:155]

In [ ]:
# based on the metadata, you can easily focus on a subset of documents
# for instance, we want to focus on all texts from the first two decades of the 17th century:

noscemus_subset = noscemus_metadata[noscemus_metadata["file_year"].between(1600, 1620)]
# to work with the subset, we need to know the IDs of the documents
ids = noscemus_subset["id"]
# Subsequently, we can load the data for each document by its ID and calculate the vocabulary of the texts:
# (depending on the size of the subset and your internet connection, this may take a while)
base_url = "https://ccs-lab.zcu.cz/noscemus_sents_data/{}.json"
subset_lemmatized_sentences = []
for id in ids: # for each work ID from our subset of IDs
    f_sents_data = json.load(io.BytesIO(requests.get(base_url.format(str(id))).content))
    sents_n = len(f_sents_data)
    for sent_data in f_sents_data:
        sent_lemmata = [t[1] for t in sent_data[3] if t[2] in ["NOUN", "VERB", "ADJ", "PROPN"]] # filter for specific POS-tags
        sent_lemmata = [re.sub(r"\W*|\d*", "", t) for t in sent_lemmata] # remove all non-alphanumeric characters
        sent_lemmata = [l for l in sent_lemmata if len(l) > 1] # remove all one-letter words
        sent_lemmata = [l.lower() for l in sent_lemmata] # lowercase all words
        subset_lemmatized_sentences.append(sent_lemmata) # add the lemmatized words from the current sentence to the overall list of lemmatized words

In [ ]:
# now you have lemmatized sentences for all texts in the subset
# let's take a look at the first few sentences:'
subset_lemmatized_sentences[:10]

In [ ]:
# you can flatten this list of lists into a single list of lemmatized words:
subset_lemmata = [lemma for sent in subset_lemmatized_sentences for lemma in sent]
# this data can be used to calculate the vocabulary of the texts:
subset_vocab = nltk.FreqDist(subset_lemmata).most_common()
subset_vocab[:100]

In [ ]:
# with lemmatized sentences, you can also immediately proceed to various kinds of co-occurrence analysis or word-embeddings.